In [1]:
import pandas as pd
import zipfile
import kaggle

In [2]:
# Download dataset from Kaggle using the kaggle API
# !kaggle datasets download -d yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring

In [3]:
# Extract the downloaded zip file
zipfile_name='fifa-21-messy-raw-dataset-for-cleaning-exploring.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [4]:
df = pd.read_csv('fifa21 raw data v2.csv')

C:\Users\zyoun\AppData\Local\Temp\ipykernel_14252\235989469.py:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('fifa21 raw data v2.csv')


In [5]:
# Explore the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

In [6]:
df.shape

(18979, 77)

In [7]:
# Convert the height and weight columns to numerical forms
# fifa21['Height'] = fifa21['Height'].str.rstrip('cm')
# fifa21['Weight'] = fifa21['Weight'].str.rstrip('kg')
# fifa21['Height'] = pd.to_numeric(fifa21['Height'])

# Noticed there's two different units of measurement in Height column
# Hard to filter out values in imperial units from metric units
# Proceed to drop rows for height values that are in feet and inches

In [8]:
# Function to check if height is in imperial units
def is_feet_inches(height_str):
    return "'" in height_str and '"' in height_str

In [9]:
# Filter rows where height is NOT in feet and inches format
fifa21 = df[~df['Height'].apply(is_feet_inches)]

In [10]:
# Check if Weight columns have data that are in lbs
print(fifa21['Weight'].str.contains('lbs').sum())

0


In [11]:
# fifa21['Height_cm'] = fifa21['Height'].str.extract(r'(\d+)cm', expand=False).astype(float)

In [12]:
# inches = fifa21['Height'].str.extract(r"(\d+)\'(\d+)\"", expand=True)
# fifa21['Height_in'] = (inches[0].astype(float) * 12 + inches[1].astype(float))

In [13]:
# Function to convert height in inches to centimeters
# def inches_to_cm(height_str):
#     feet, inches = map(int, height_str.split("'"))
#     total_inches = feet * 12 + inches
#     return total_inches * 2.54

In [14]:
# 2. Split the 'Joined' column into Year, Month, Day
fifa21[['Month','Day','Year']] = fifa21['Joined'].str.split(r" |, ", expand=True)

C:\Users\zyoun\AppData\Local\Temp\ipykernel_14252\2686359746.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fifa21[['Month','Day','Year']] = fifa21['Joined'].str.split(r" |, ", expand=True)
C:\Users\zyoun\AppData\Local\Temp\ipykernel_14252\2686359746.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fifa21[['Month','Day','Year']] = fifa21['Joined'].str.split(r" |, ", expand=True)
C:\Users\zyoun\AppData\Local\Temp\ipykernel_14252\2686359746.py:2: SettingWithCopyWarning: 
A value is trying to be set on

In [15]:
# Renaming Columns
fifa21 = fifa21.rename(columns={
    'Joined' : 'Joined_Date'
})

In [16]:
# Convert Joined datatype from str to DateTime 
fifa21['Joined_Date'] = pd.to_datetime(fifa21['Joined_Date'])

In [17]:
# Convert Value, Wage, and Release Clause to numeric columns

# fifa21['Value'] = fifa21['Value'].str.replace('€', '')
# fifa21['Wage'] = fifa21['Wage'].str.replace('€', '')
# fifa21['Release Clause'] = fifa21['Release Clause'].str.replace('€', '')

fifa21[['Value', 'Wage', 'Release Clause']] = fifa21[['Value', 'Wage', 'Release Clause']].apply(lambda x: x.str.replace('€', ''))

In [19]:
fifa21['Value'] = fifa21['Value'].apply(lambda x: pd.to_numeric(x.replace('K','')) * 1000 if 'K' in x
                             else pd.to_numeric(x.replace('M','')) * 1000000)
fifa21['Release Clause'] = fifa21['Release Clause'].apply(lambda x: pd.to_numeric(x.replace('K','')) * 1000 if 'K' in x
                                                                                  else pd.to_numeric(x.replace('M','')) * 1000000)
fifa21['Wage'] = fifa21['Wage'].apply(lambda x: pd.to_numeric(x.replace('K','')) * 1000 if 'K' in x
                                      else pd.to_numeric((x)))

fifa21[['Value', 'Release Clause']]=fifa21[['Value', 'Release Clause']].astype(int)

In [41]:
# Strip \n new lines in Club column
fifa21['Club']=fifa21['Club'].str.strip()

In [ ]:
# Strip ★ and convert to columns to numeric type
fifa21[['W/F','SM','IR']] = fifa21[['W/F','SM','IR']].apply(lambda x: pd.to_numeric(x.str.strip("★")))


In [58]:
pd.set_option("display.max_columns", None)

In [60]:
fifa21.drop_duplicates()

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined_Date,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,Month,Day,Year
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,FC Barcelona,2004 ~ 2021,"RW, ST, CF",170cm,72kg,Left,93,RW,2004-07-01,NaN,103500000,560000,138400000,429,85,95,70,91,88,470,96,93,94,91,96,451,91,80,91,94,95,389,86,68,72,69,94,347,44,40,93,95,75,96,91,32,35,24,54,6,11,15,14,8,2231,466,4,4,Medium,Low,5,85,92,91,95,38,65,771,Jul,1,2004
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,Juventus,2018 ~ 2022,"ST, LW",187cm,83kg,Right,92,ST,2018-07-10,NaN,63000000,220000,75900000,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,4,5,High,Low,5,89,93,81,89,35,77,562,Jul,10,2018
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,Atlético Madrid,2014 ~ 2023,GK,188cm,87kg,Right,91,GK,2014-07-16,NaN,120000000,125000,159400000,95,13,11,15,43,13,109,12,13,14,40,30,307,43,60,67,88,49,268,59,78,41,78,12,140,34,19,11,65,11,68,57,27,12,18,437,87,92,78,90,90,1413,489,3,1,Medium,Medium,3,87,92,78,90,52,90,150,Jul,16,2014
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,Manchester City,2015 ~ 2023,"CAM, CM",181cm,70kg,Right,91,CAM,2015-08-30,NaN,129000000,370000,161000000,407,94,82,55,94,82,441,88,85,83,93,92,398,77,76,78,91,76,408,91,63,89,74,91,408,76,66,88,94,84,91,186,68,65,53,56,15,13,5,10,13,2304,485,5,4,High,High,4,76,86,93,88,64,78,207,Aug,30,2015
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,Paris Saint-Germain,2017 ~ 2022,"LW, CAM",175cm,68kg,Right,91,LW,2017-08-03,NaN,132000000,270000,166500000,408,85,87,62,87,87,448,95,88,89,81,95,453,94,89,96,91,83,357,80,62,81,50,84,356,51,36,87,90,92,93,94,35,30,29,59,9,9,15,15,11,2175,451,5,5,High,Medium,5,91,85,86,94,36,59,595,Aug,3,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18974,247223,Xia Ao,Ao Xia,https://cdn.sofifa.com/players/247/223/21_60.png,http://sofifa.com/player/247223/ao-xia/210006/,China PR,21,47,55,Wuhan Zall,2018 ~ 2022,CB,178cm,66kg,Right,49,CB,2018-07-13,NaN,100000,1000,70000,145,23,26,43,26,27,142,27,23,21,29,42,294,68,60,69,46,51,221,36,57,54,50,24,192,48,50,28,28,38,44,147,45,52,50,45,7,8,5,14,11,1186,255,2,2,Medium,Medium,1,64,28,26,38,48,51,NaN,Jul,13,2018
18975,258760,B. Hough,Ben Hough,https://cdn.sofifa.com/players/258/760/21_60.png,http://sofifa.com/player/258760/ben-hough/210006/,England,17,47,67,Oldham Athletic,2020 ~ 2021,CM,175cm,65kg,Right,51,CAM,2020-08-01,NaN,130000,500,165000,211,38,42,40,56,35,219,46,40,35,50,48,305,63,64,61,51,66,226,48,58,43,47,30,193,40,23,47,47,36,38,116,32